In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 150)
import seaborn as sns
sns.set_theme(color_codes=True)
from scipy import stats
import statsmodels.api as sm
import numpy as np
import json

/opt/homebrew/lib/python3.9/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def load_data(folder_address = "./Data/Questionnaire Data/" ):
    list_of_data_names = os.listdir(folder_address)
    data = []
    for i in list_of_data_names:
        if(i != ".DS_Store"):
            df = pd.read_csv(folder_address + i)
            data.append(df)

    return pd.concat(data)
            

    

In [3]:
df = load_data()

In [4]:
len(df.mturkID.unique())

40

In [5]:
questionnaire_raw_data = df
# questionnaire_raw_data = questionnaire_raw_data.dropna(axis=0, how='any')
questionnaire_respones = questionnaire_raw_data[questionnaire_raw_data.trial_type.isin(['table-completion', 'custom-form'])][['mturkID', 'experiment_id', 'trial_type', 'id', 'responses']]


In [6]:
len(questionnaire_respones.mturkID.unique())

40

In [7]:

def clean_table_questions(x):
    dic = {}
    a = json.loads(x)['responses']
    for i in a:
        dic[i] = a[i]['choices'][0]
    return dic

def clean_form_questions(x):
    dic = {}
    for i in json.loads(x):
        dic[i['id']] = i['value']
    return dic

def clean_questions(x,type):
    if(type == 'table-completion'):
        return clean_table_questions(x)
    elif(type == 'custom-form'):
        return clean_form_questions(x)
    else:
        return x

In [8]:
questionnaire_respones['clean_responses'] =  questionnaire_respones.apply(lambda x: clean_questions(x['responses'],x['trial_type']), axis=1)

In [9]:
questionnaire_respones.clean_responses = questionnaire_respones.clean_responses.apply(lambda x: list(x.items()))

In [10]:
a = questionnaire_respones.groupby('mturkID').agg({'clean_responses': 'sum'}).reset_index()

In [11]:
a['clean_responses_dict'] = a.clean_responses.apply(lambda x: dict(x))

In [12]:
df_dict = {}

for i in a.iterrows():
    df_dict[(i[1]['mturkID'])] = i[1]['clean_responses_dict']

In [13]:
df = pd.DataFrame(df_dict).transpose().reset_index()

In [14]:
df

,index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,weight,heightfeet,heightinch,pet-1,pet-type,pet-number,pet-timeperiod,pet-0
0,A12FTSX85NQ8N9,1,1,1,1,1,0,0,0,1,1,3,4,1,1,1,0,0,0,0,1,0,1,1,0,3,4,1,1,180,5,7,yes,Cat,2,16,NaN
1,A1CY7IOJ9YH136,1,2,6,3,2,2,1,0,0,1,6,1,0,0,0,0,0,0,0,1,0,3,3,1,3,4,3,3,140,5,2,yes,dog,2,16 years,NaN
2,A1DZMZTXWOM9MR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,205,6,0,yes,cat,1,7,NaN
3,A1FVXS8IM5QYO8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120,5,6,yes,Dog,1,4,NaN
4,A1MJVTR0PCKBWW,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,125,5,5,yes,dog,2,6,NaN
5,A1PAY3X73PQ16S,0,0,6,0,3,0,1,1,2,3,6,3,4,4,4,0,0,0,1,1,2,6,6,3,6,6,6,6,204,5,4,yes,Cat,1,16,NaN
6,A1R8A8BK2VN7RH,2,0,2,2,3,6,0,0,1,2,4,4,0,0,0,0,5,3,0,0,0,4,4,3,4,5,5,6,130,5,6,yes,cat,1,3,NaN
7,A1UDOSYE75OYYI,1,0,1,0,0,0,0,0,0,0,0,0,5,2,0,0,0,5,0,0,0,1,1,1,1,1,0,1,145,5,10,NaN,NaN,NaN,NaN,no
8,A2196WCNDZULFS,0,0,0,0,0,0,0,0,0,0,6,3,0,0,0,0,0,0,0,0,0,1,1,0,6,6,3,5,234,5,5,yes,"cats, dogs",7,20,NaN
9,A24CG5BTB8KQ39,1,0,0,0,0,1,0,0,0,0,1,1,2,0,0,0,0,0,0,1,0,1,0,1,2,2,2,2,222,5,4,yes,dog,1,7,NaN


In [15]:
df.to_csv('./Data/cleaned_questionnaire_data.csv')